##### This notebook contains code to conduct commonality analysis

In [1]:
#load libraries
import pandas as pd
import numpy as np
import glob
import os

from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from scipy.stats import linregress

In [2]:
#function return r2 from linear regression between feature(s) and target
def get_r2(X, y):
    if len(X.shape)<2:
        r2 = LinearRegression().fit(StandardScaler().fit_transform(X.values.reshape(-1, 1)), y).score(StandardScaler().fit_transform(X.values.reshape(-1, 1)), y)
    else:
        r2 = LinearRegression().fit(StandardScaler().fit_transform(X.values), y).score(StandardScaler().fit_transform(X.values), y)
    return r2
        

In [3]:
#path to output directory 
path = '/media/data/HCPAging/data/MLTablesMultCon/output_5cv_sexAdj_noStdTarg_STDstackFeatures/'

In [4]:
#load dataframes

#age prediction from test
df_age = pd.read_csv(path+'output_5cv_sexAdj_noStdTarg_STDstackFeatures_interview_age/all_fold_y-pred_y-obs.csv', index_col=0)
#age prediction from train
df_age_train = pd.read_csv(path+'output_5cv_sexAdj_noStdTarg_STDstackFeatures_interview_age/all_fold_y-pred_y-obs_train.csv', index_col=0)
#fluid cognition prediction
df_cog = pd.read_csv(path+'output_5cv_sexAdj_noStdTarg_STDstackFeatures_nih_fluidcogcomp_unadjusted/all_fold_y-pred_y-obs.csv', index_col=0)

In [5]:
#define original targets and predicted values for each type
cog_orig = df_cog['y_real']
cog_pred = df_cog.iloc[:, :-2]

age_orig = df_age['y_real']
age_pred = df_age.iloc[:, :-2]

In [6]:
#calculate brain age corrected

dct_corrected ={}

for fold in sorted(set(df_age_train['fold'])):
    dct_corrected[fold] ={}
    
    df_pred = df_age_train[df_age_train['fold']==fold].iloc[:, :-2]
    df_orig = df_age_train[df_age_train['fold']==fold]['y_real']
    df_test = df_age[df_age['fold']==fold].iloc[:, :-2]
    
    for col in df_pred.columns:
        slope, intercept, _, _, _, = linregress(df_orig, df_pred[col])
        
        out = (df_test[col] - intercept)/slope
        
        
        dct_corrected[fold][col]= out
        
age_pred_cor = pd.DataFrame(dct_corrected[0])
for fold in sorted(set(df_age_train['fold']))[1:]:
    age_pred_cor = pd.concat([age_pred_cor, pd.DataFrame(dct_corrected[fold])], axis=0)

In [7]:
#calculate brain age gap, usual and corrected

age_gap = pd.DataFrame()
for col in age_pred.columns:
    age_gap[col] = age_orig - age_pred[col]


age_gap_cor = pd.DataFrame()
for col in age_pred_cor.columns:
    age_gap_cor[col] = age_orig - age_pred_cor[col]

In [8]:
# one predictor
#cognition                              cog_orig        1
#age                                    age_orig        1
#brain age                              age_pred        26
#brain age gap (delta)                  age_gap         26
#corrected brain age                    age_pred_cor    26
#corrected brain age gap (delta)        age_gap_cor     26

In [9]:
#Predicting cognition with age

#one predictor (one of brain age indexes)

dct_r2_one = {}
dct_corR_one = {}
dct_corPval_one = {}

for dset, dset_name in zip([age_orig, age_pred, age_gap, age_pred_cor, age_gap_cor], 
                           ['age_orig', 'age_pred', 'age_gap', 'age_pred_cor', 'age_gap_cor']):  
    #
    
    if len(dset.shape)<2:
        
        dct_r2_one[dset_name] = get_r2(dset, cog_orig)
        
    else:
        
        dct_r2_one[dset_name] = {}
        dct_corR_one[dset_name] = {}
        dct_corPval_one[dset_name] = {}
        
        for col in dset.columns:
            
            dct_r2_one[dset_name][col] = get_r2(dset[col], cog_orig)
            dct_corR_one[dset_name][col], dct_corPval_one[dset_name][col] = pearsonr(dset[col], cog_orig)
        
        dct_r2_one[dset_name] = pd.Series(dct_r2_one[dset_name])
        dct_corR_one[dset_name] = pd.Series(dct_corR_one[dset_name])
        dct_corPval_one[dset_name] = pd.Series(dct_corPval_one[dset_name])

print('Predicting cognition with age')
print(' ')
print('age_orig', dct_r2_one['age_orig'])
print('age_orig correlation with cog', pearsonr(age_orig, cog_orig))

df_r2_one = pd.DataFrame()
df_corR_one = pd.DataFrame()
df_corPval_one = pd.DataFrame()

for key in list(dct_r2_one.keys())[1:]:
    df_r2_one['r2_1_'+key] = dct_r2_one[key]
    df_corR_one['R_1_'+key] = dct_corR_one[key]
    df_corPval_one['Pval_1_'+key] = dct_corPval_one[key]

print(' ')    
print('R2')    
display(df_r2_one.round(6))
print(' ') 
print('Correlation')
display(df_corR_one.round(6))
print(' ') 
print('Pvalue for correlation')
display(df_corPval_one.round(6))

Predicting cognition with age
 
age_orig 0.32073185105582647
age_orig correlation with cog (-0.5663319265729474, 4.35360066270803e-44)
 
R2


,r2_1_age_pred,r2_1_age_gap,r2_1_age_pred_cor,r2_1_age_gap_cor
carit1,0.052275,0.278114,0.056524,0.036141
carit3,0.101934,0.228756,0.105562,0.028827
carit4,0.051659,0.267819,0.052987,0.022561
face1,0.118230,0.199998,0.114327,0.012302
face2,0.115699,0.199401,0.118205,0.014101
face3,0.113068,0.223987,0.115074,0.006080
face4,0.043832,0.272638,0.044384,0.026787
face5,0.166395,0.165657,0.166601,0.007732
face6,0.175158,0.166654,0.166113,0.002471
vism,0.129792,0.195600,0.132140,0.012664


 
Correlation


,R_1_age_pred,R_1_age_gap,R_1_age_pred_cor,R_1_age_gap_cor
carit1,-0.228638,-0.527366,-0.237748,-0.190108
carit3,-0.319272,-0.478285,-0.324903,-0.169785
carit4,-0.227287,-0.517513,-0.230190,-0.150203
face1,-0.343845,-0.447212,-0.338122,-0.110916
face2,-0.340146,-0.446543,-0.343810,-0.118749
face3,-0.336255,-0.473272,-0.339226,-0.077971
face4,-0.209361,-0.522147,-0.210674,-0.163668
face5,-0.407916,-0.407010,-0.408168,-0.087930
face6,-0.418519,-0.408233,-0.407569,-0.049710
vism,-0.360266,-0.442267,-0.363511,-0.112533


 
Pvalue for correlation


,Pval_1_age_pred,Pval_1_age_gap,Pval_1_age_pred_cor,Pval_1_age_gap_cor
carit1,0.000000,0.0,0.000000,0.000017
carit3,0.000000,0.0,0.000000,0.000128
carit4,0.000000,0.0,0.000000,0.000717
face1,0.000000,0.0,0.000000,0.012718
face2,0.000000,0.0,0.000000,0.007613
face3,0.000000,0.0,0.000000,0.080331
face4,0.000002,0.0,0.000002,0.000224
face5,0.000000,0.0,0.000000,0.048503
face6,0.000000,0.0,0.000000,0.265320
vism,0.000000,0.0,0.000000,0.011468


In [10]:
#Predicting cognition with age and predicted age index

#two predictors (one of brain age indexes and real age)

dct_r2_two = {}
dct_u1_two = {}
dct_u2_two = {}
dct_c_two = {}

for dset, dset_name in zip([age_pred, age_gap, age_pred_cor, age_gap_cor], 
                           ['age_pred', 'age_gap', 'age_pred_cor', 'age_gap_cor']):  
    #
    dct_r2_two[dset_name] = {}
    dct_u1_two[dset_name] = {}
    dct_u2_two[dset_name] = {}
    dct_c_two[dset_name] = {}
    
    for col in dset.columns:
        #
        r2_i = get_r2(dset[col], cog_orig)
        r2_j = get_r2(age_orig, cog_orig)
        r2_ij = get_r2(pd.concat([dset[col], age_orig], axis=1), cog_orig)

        u_i = r2_ij - r2_j
        u_j = r2_ij - r2_i

        c_ij = r2_ij - u_i - u_j
        
        dct_r2_two[dset_name][col] = r2_ij
        dct_u1_two[dset_name][col] = u_i
        dct_u2_two[dset_name][col] = u_j
        dct_c_two[dset_name][col] = c_ij
        
    dct_r2_two[dset_name] = pd.Series(dct_r2_two[dset_name])
    dct_u1_two[dset_name] = pd.Series(dct_u1_two[dset_name])
    dct_u2_two[dset_name] = pd.Series(dct_u2_two[dset_name])
    dct_c_two[dset_name] = pd.Series(dct_c_two[dset_name])


df_r2_two = pd.DataFrame(dct_r2_two)
df_r2_two.columns = ['r2_2_'+i+"+age-orig" for i in df_r2_two.columns]
df_u1_two = pd.DataFrame(dct_u1_two)
df_u1_two.columns = ['U_2_'+i for i in df_u1_two.columns]

df_u2_two = pd.DataFrame(dct_u2_two)
df_u2_two.columns = ['U_2_age-orig_('+i+')' for i in df_u2_two.columns]

df_c_two = pd.DataFrame(dct_c_two)
df_c_two.columns = ['C_2_'+i+'+age-orig' for i in df_c_two.columns]

print('Two predictors for cognition prediction')
print('R2, uniq R2, common R2')
display(pd.concat([df_r2_two, df_u1_two, df_u2_two, df_c_two], axis=1) )

Two predictors for cognition prediction
R2, uniq R2, common R2


,r2_2_age_pred+age-orig,r2_2_age_gap+age-orig,r2_2_age_pred_cor+age-orig,r2_2_age_gap_cor+age-orig,U_2_age_pred,U_2_age_gap,U_2_age_pred_cor,U_2_age_gap_cor,U_2_age-orig_(age_pred),U_2_age-orig_(age_gap),U_2_age-orig_(age_pred_cor),U_2_age-orig_(age_gap_cor),C_2_age_pred+age-orig,C_2_age_gap+age-orig,C_2_age_pred_cor+age-orig,C_2_age_gap_cor+age-orig
carit1,0.323623,0.323623,0.323603,0.323603,0.002891,0.002891,0.002871,0.002871,0.271348,0.045509,0.267078,0.287461,0.049384,0.275223,0.053653,0.033270
carit3,0.322518,0.322518,0.322415,0.322415,0.001786,0.001786,0.001683,0.001683,0.220584,0.093762,0.216853,0.293588,0.100148,0.226970,0.103879,0.027144
carit4,0.320867,0.320867,0.320890,0.320890,0.000135,0.000135,0.000158,0.000158,0.269208,0.053048,0.267902,0.298329,0.051524,0.267684,0.052830,0.022403
face1,0.324921,0.324921,0.323603,0.323603,0.004189,0.004189,0.002871,0.002871,0.206691,0.124923,0.209276,0.311300,0.114041,0.195809,0.111456,0.009432
face2,0.322564,0.322564,0.322729,0.322729,0.001832,0.001832,0.001997,0.001997,0.206865,0.123163,0.204524,0.308627,0.113867,0.197569,0.116208,0.012105
face3,0.328956,0.328956,0.329435,0.329435,0.008225,0.008225,0.008704,0.008704,0.215889,0.104970,0.214361,0.323356,0.104843,0.215762,0.106371,-0.002624
face4,0.320826,0.320826,0.321052,0.321052,0.000094,0.000094,0.000320,0.000320,0.276994,0.048188,0.276668,0.294265,0.043738,0.272544,0.044064,0.026467
face5,0.328370,0.328370,0.327689,0.327689,0.007638,0.007638,0.006958,0.006958,0.161975,0.162713,0.161088,0.319958,0.158757,0.158019,0.159644,0.000774
face6,0.336832,0.336832,0.332534,0.332534,0.016100,0.016100,0.011802,0.011802,0.161673,0.170178,0.166421,0.330063,0.159058,0.150554,0.154311,-0.009331
vism,0.325712,0.325712,0.325498,0.325498,0.004981,0.004981,0.004766,0.004766,0.195921,0.130112,0.193358,0.312834,0.124811,0.190619,0.127374,0.007897


In [12]:
#Predicting cognition with age, predicted age index and predicted cognition

#three predictors (one of brain age indexes, real age, predicted cognition)

dct_r2_one_cog = {}
dct_r2_two_age_cogpr = {}
dct_r2_two_agepr_cogpr = {}
dct_r2_three = {}

dct_u1_three = {}
dct_u2_three = {}
dct_u3_three = {}

dct_c1_three = {}
dct_c2_three = {}
dct_c3_three = {}
dct_c4_three = {}

for dset, dset_name in zip([age_pred, age_gap, age_pred_cor, age_gap_cor], 
                           ['age_pred', 'age_gap', 'age_pred_cor', 'age_gap_cor']):  
    #
    dct_r2_one_cog[dset_name] = {}
    dct_r2_two_age_cogpr[dset_name] = {}
    dct_r2_two_agepr_cogpr[dset_name] = {}
    dct_r2_three[dset_name] = {}

    dct_u1_three[dset_name] = {}
    dct_u2_three[dset_name] = {}
    dct_u3_three[dset_name] = {}

    dct_c1_three[dset_name] = {}
    dct_c2_three[dset_name] = {}
    dct_c3_three[dset_name] = {}
    dct_c4_three[dset_name] = {}
    
    for col in dset.columns:
        #
        r2_i = get_r2( dset[col], cog_orig)
        r2_j = get_r2( age_orig, cog_orig)
        r2_k = get_r2( cog_pred[col], cog_orig) #

        r2_jk = get_r2( pd.concat([ age_orig, cog_pred[col]],axis=1), cog_orig) #
        r2_ik = get_r2( pd.concat([ dset[col], cog_pred[col]],axis=1), cog_orig) #
        r2_ij = get_r2( pd.concat([ dset[col], age_orig],axis=1), cog_orig)

        r2_ijk = get_r2( pd.concat([ dset[col], age_orig, cog_pred[col] ],axis=1), cog_orig) #

        u_i = r2_ijk - r2_jk # uniq for age_predicted
        u_j = r2_ijk - r2_ik # uniq for age
        u_k = r2_ijk - r2_ij # uniq for cog predicted

        c_ij = r2_ik + r2_jk - r2_k - r2_ijk
        c_ik = r2_ij + r2_jk - r2_j - r2_ijk
        c_jk = r2_ij + r2_ik - r2_i - r2_ijk
        c_ijk = r2_i + r2_j + r2_k - r2_ij - r2_ik - r2_jk + r2_ijk
        
        
        dct_r2_one_cog[dset_name][col] = r2_k
        dct_r2_two_age_cogpr[dset_name][col] = r2_jk
        dct_r2_two_agepr_cogpr[dset_name][col] = r2_ik
        dct_r2_three[dset_name][col] = r2_ijk

        dct_u1_three[dset_name][col] = u_i
        dct_u2_three[dset_name][col] = u_j
        dct_u3_three[dset_name][col] = u_k

        dct_c1_three[dset_name][col] = c_ij
        dct_c2_three[dset_name][col] = c_ik
        dct_c3_three[dset_name][col] = c_jk
        dct_c4_three[dset_name][col] = c_ijk
    
    dct_r2_one_cog[dset_name] = pd.Series(dct_r2_one_cog[dset_name])
    dct_r2_two_age_cogpr[dset_name] = pd.Series(dct_r2_two_age_cogpr[dset_name])
    dct_r2_two_agepr_cogpr[dset_name] = pd.Series(dct_r2_two_agepr_cogpr[dset_name])
    dct_r2_three[dset_name] = pd.Series(dct_r2_three[dset_name])

    dct_u1_three[dset_name] = pd.Series(dct_u1_three[dset_name])
    dct_u2_three[dset_name] = pd.Series(dct_u2_three[dset_name])
    dct_u3_three[dset_name] = pd.Series(dct_u3_three[dset_name])

    dct_c1_three[dset_name] = pd.Series(dct_c1_three[dset_name])
    dct_c2_three[dset_name] = pd.Series(dct_c2_three[dset_name])
    dct_c3_three[dset_name] = pd.Series(dct_c3_three[dset_name])
    dct_c4_three[dset_name] = pd.Series(dct_c4_three[dset_name])

df_r2_one_cog = pd.DataFrame(dct_r2_one_cog['age_pred'])
df_r2_one_cog.columns = ['r2_1_cog_pred']
df_r2_two_age_cogpr = pd.DataFrame(dct_r2_two_age_cogpr['age_pred'])
df_r2_two_age_cogpr.columns = ['r2_3_2_age-orig+cog-pred']
df_r2_two_agepr_cogpr = pd.DataFrame(dct_r2_two_agepr_cogpr)
df_r2_two_agepr_cogpr.columns = ['r2_3_2_'+i+'+cog-pred' for i in df_r2_two_agepr_cogpr.columns]
df_r2_three = pd.DataFrame(dct_r2_three)
df_r2_three.columns = ['r2_3_'+i+'+age+cog-pred' for i in df_r2_three.columns]

df_u1_three = pd.DataFrame(dct_u1_three)
df_u1_three.columns = ['U_3_'+i for i in df_u1_three.columns]
df_u2_three = pd.DataFrame(dct_u2_three)
df_u2_three.columns = ['U_3_age-orig_('+i+')' for i in df_u2_three.columns]
df_u3_three = pd.DataFrame(dct_u3_three)
df_u3_three.columns = ['U_3_cog-pred_('+i+')' for i in df_u3_three.columns]

df_c1_three = pd.DataFrame(dct_c1_three)
df_c1_three.columns = ['C_3_2_'+i+'+age-orig' for i in df_c1_three.columns]
df_c2_three = pd.DataFrame(dct_c2_three)
df_c2_three.columns = ['C_3_2_'+i+'+cog-pred' for i in df_c2_three.columns]
df_c3_three = pd.DataFrame(dct_c3_three)
df_c3_three.columns = ['C_3_2_age-orig+cog-pred_('+i+')' for i in df_c3_three.columns]
df_c4_three = pd.DataFrame(dct_c4_three)
df_c4_three.columns = ['C_3_'+i+'+age+cog-pred' for i in df_c4_three.columns]


print('Three predictors for cognition prediction')
print('R2, uniq R2, common R2')
display(pd.concat([df_r2_one_cog, df_r2_two_age_cogpr, df_r2_two_agepr_cogpr, df_r2_three,
                   df_u1_three, df_u2_three, df_u3_three, 
                   df_c1_three, df_c2_three, df_c3_three, df_c4_three], axis=1))

Three predictors for cognition prediction
R2, uniq R2, common R2


,r2_1_cog_pred,r2_3_2_age-orig+cog-pred,r2_3_2_age_pred+cog-pred,r2_3_2_age_gap+cog-pred,r2_3_2_age_pred_cor+cog-pred,r2_3_2_age_gap_cor+cog-pred,r2_3_age_pred+age+cog-pred,r2_3_age_gap+age+cog-pred,r2_3_age_pred_cor+age+cog-pred,r2_3_age_gap_cor+age+cog-pred,...,C_3_2_age_pred_cor+cog-pred,C_3_2_age_gap_cor+cog-pred,C_3_2_age-orig+cog-pred_(age_pred),C_3_2_age-orig+cog-pred_(age_gap),C_3_2_age-orig+cog-pred_(age_pred_cor),C_3_2_age-orig+cog-pred_(age_gap_cor),C_3_age_pred+age+cog-pred,C_3_age_gap+age+cog-pred,C_3_age_pred_cor+age+cog-pred,C_3_age_gap_cor+age+cog-pred
carit1,0.031342,0.324699,0.054341,0.304384,0.058259,0.118486,0.325108,0.325108,0.325132,0.325132,...,0.002438,0.002438,0.000581,0.024785,0.000206,0.080815,0.026793,0.002589,0.027168,-0.053441
carit3,0.090492,0.329620,0.113506,0.305662,0.116203,0.193529,0.330178,0.330178,0.330220,0.330220,...,0.001083,0.001083,0.003911,0.069246,0.002836,0.156896,0.077692,0.012358,0.078767,-0.075293
carit4,0.058000,0.327590,0.068259,0.313653,0.069583,0.135741,0.329422,0.329422,0.329163,0.329163,...,-0.001415,-0.001415,0.008044,0.037279,0.008323,0.104907,0.043098,0.013863,0.042820,-0.053765
face1,0.171927,0.364096,0.176301,0.365313,0.175386,0.287769,0.373047,0.373047,0.375829,0.375829,...,-0.008862,-0.008862,0.009945,0.117189,0.008833,0.223240,0.118617,0.011373,0.119729,-0.094678
face2,0.182768,0.366092,0.184428,0.379790,0.184889,0.309377,0.383312,0.383312,0.382658,0.382658,...,-0.014570,-0.014570,0.007980,0.119641,0.006754,0.235346,0.129427,0.017767,0.130653,-0.097938
face3,0.192793,0.388379,0.194318,0.397348,0.194816,0.301575,0.399314,0.399314,0.398258,0.398258,...,-0.001175,-0.001175,0.010893,0.103004,0.010920,0.226673,0.114253,0.022142,0.114226,-0.101527
face4,0.077471,0.333081,0.077944,0.342830,0.078576,0.198560,0.345195,0.345195,0.345634,0.345634,...,-0.012234,-0.012234,0.009742,0.045823,0.009610,0.147191,0.055380,0.019299,0.055512,-0.082069
face5,0.242093,0.390915,0.246551,0.398856,0.246583,0.349168,0.403758,0.403758,0.404999,0.404999,...,-0.007126,-0.007126,0.004768,0.157810,0.002673,0.264127,0.167142,0.014100,0.169237,-0.092217
face6,0.244930,0.400871,0.251011,0.401657,0.249501,0.342595,0.408096,0.408096,0.411520,0.411520,...,0.001153,0.001153,0.004588,0.163738,0.004402,0.261137,0.160202,0.001052,0.160389,-0.096347
vism,0.166127,0.365573,0.175292,0.366240,0.176300,0.283919,0.374455,0.374455,0.374763,0.374763,...,-0.004423,-0.004423,-0.003242,0.121898,-0.005105,0.221991,0.124527,-0.000613,0.126391,-0.100705


In [13]:
#add extra columns from ML

dct_age_brain={}
for col in age_pred.columns:
    dct_age_brain[col] = get_r2( age_pred[col], age_orig)
    
age_brain = pd.Series(dct_age_brain)
age_brain.name = 'r2_AGE_brain'

cog_brain = df_r2_one_cog['r2_1_cog_pred']
cog_brain.name = 'r2_COG_brain'

In [14]:
#assemble into one big table and save

big_table = pd.concat([cog_brain, age_brain, df_r2_one, 
                       df_r2_two, df_u1_two, df_u2_two, df_c_two,
                       df_r2_two_age_cogpr, df_r2_two_agepr_cogpr, df_r2_three,
                       df_u1_three, df_u2_three, df_u3_three, 
                       df_c1_three, df_c2_three, df_c3_three, df_c4_three], axis=1)
big_table.index = ['carit1-allNoGo-allGo', 'carit3-allGo', 'carit4-allNoGo', 'face1-encoding', 'face2-recall', 'face3-distractor', 'face4-encoding-recall',
                   'face5-encoding-distractor', 'face6-recall-distractor', 'VisualMotor', 'carit_FC', 'face_FC', 'vism_FC', 'cort_thickness',
                   'cort_surface', 'subcortex', 'Total_Brain_Volume', 'Resting_State_FC_75PC', 
                   'All-Modality_con+FC(set1)', 'Task_con(set2)', 'Non-Task(set3)', 
                   'Task_FC(set4)', 'Task_con+FC(set5)', 'All-modality_con(set6)', 
                   'All-modality_FC(set7)', 'FC_rest_task(set8)']
display(big_table)

#save
big_table.to_csv(path+'commonality_cog_fluid.csv')

,r2_COG_brain,r2_AGE_brain,r2_1_age_pred,r2_1_age_gap,r2_1_age_pred_cor,r2_1_age_gap_cor,r2_2_age_pred+age-orig,r2_2_age_gap+age-orig,r2_2_age_pred_cor+age-orig,r2_2_age_gap_cor+age-orig,...,C_3_2_age_pred_cor+cog-pred,C_3_2_age_gap_cor+cog-pred,C_3_2_age-orig+cog-pred_(age_pred),C_3_2_age-orig+cog-pred_(age_gap),C_3_2_age-orig+cog-pred_(age_pred_cor),C_3_2_age-orig+cog-pred_(age_gap_cor),C_3_age_pred+age+cog-pred,C_3_age_gap+age+cog-pred,C_3_age_pred_cor+age+cog-pred,C_3_age_gap_cor+age+cog-pred
carit1-allNoGo-allGo,0.031342,0.098319,0.052275,0.278114,0.056524,0.036141,0.323623,0.323623,0.323603,0.323603,...,0.002438,0.002438,0.000581,0.024785,0.000206,0.080815,0.026793,0.002589,0.027168,-0.053441
carit3-allGo,0.090492,0.249086,0.101934,0.228756,0.105562,0.028827,0.322518,0.322518,0.322415,0.322415,...,0.001083,0.001083,0.003911,0.069246,0.002836,0.156896,0.077692,0.012358,0.078767,-0.075293
carit4-allNoGo,0.058000,0.146194,0.051659,0.267819,0.052987,0.022561,0.320867,0.320867,0.320890,0.320890,...,-0.001415,-0.001415,0.008044,0.037279,0.008323,0.104907,0.043098,0.013863,0.042820,-0.053765
face1-encoding,0.171927,0.258833,0.118230,0.199998,0.114327,0.012302,0.324921,0.324921,0.323603,0.323603,...,-0.008862,-0.008862,0.009945,0.117189,0.008833,0.223240,0.118617,0.011373,0.119729,-0.094678
face2-recall,0.182768,0.288199,0.115699,0.199401,0.118205,0.014101,0.322564,0.322564,0.322729,0.322729,...,-0.014570,-0.014570,0.007980,0.119641,0.006754,0.235346,0.129427,0.017767,0.130653,-0.097938
face3-distractor,0.192793,0.203222,0.113068,0.223987,0.115074,0.006080,0.328956,0.328956,0.329435,0.329435,...,-0.001175,-0.001175,0.010893,0.103004,0.010920,0.226673,0.114253,0.022142,0.114226,-0.101527
face4-encoding-recall,0.077471,0.148571,0.043832,0.272638,0.044384,0.026787,0.320826,0.320826,0.321052,0.321052,...,-0.012234,-0.012234,0.009742,0.045823,0.009610,0.147191,0.055380,0.019299,0.055512,-0.082069
face5-encoding-distractor,0.242093,0.355701,0.166395,0.165657,0.166601,0.007732,0.328370,0.328370,0.327689,0.327689,...,-0.007126,-0.007126,0.004768,0.157810,0.002673,0.264127,0.167142,0.014100,0.169237,-0.092217
face6-recall-distractor,0.244930,0.304936,0.175158,0.166654,0.166113,0.002471,0.336832,0.336832,0.332534,0.332534,...,0.001153,0.001153,0.004588,0.163738,0.004402,0.261137,0.160202,0.001052,0.160389,-0.096347
VisualMotor,0.166127,0.281437,0.129792,0.195600,0.132140,0.012664,0.325712,0.325712,0.325498,0.325498,...,-0.004423,-0.004423,-0.003242,0.121898,-0.005105,0.221991,0.124527,-0.000613,0.126391,-0.100705


In [16]:
#save extra tables

#save table with r2 only (not uniq or common part of r2)
pd.concat([df_r2_one, df_r2_two, df_r2_three], axis=1).to_csv(path+'r2_ages_cog_fluid.csv')

df_corR_one.to_csv(path+'correlation_r_agepreds_cog_fluid.csv')

df_corPval_one.to_csv(path+'correlation_pval_agepreds_cog_fluid.csv')

'/media/data/HCPAging/data/MLTablesMultCon/output_5cv_sexAdj_noStdTarg_STDstackFeatures/'